In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['embeddings', 'sample_submission.csv', 'test.csv', 'train.csv']


In [2]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")
print("Train shape : ",train_df.shape)
print("Test shape : ",test_df.shape)

Train shape :  (1306122, 3)
Test shape :  (375806, 2)


In [3]:
print('Max word length of questions in train is {0:.0f}.'.format(np.max(train_df['question_text'].apply(lambda x: len(x.split())))))
print('Max word length of questions in test is {0:.0f}.'.format(np.max(test_df['question_text'].apply(lambda x: len(x.split())))))

Max word length of questions in train is 134.
Max word length of questions in test is 87.


In [4]:
print('Average character length of questions in train is {0:.0f}.'.format(np.mean(train_df['question_text'].apply(lambda x: len(x)))))
print('Average character length of questions in test is {0:.0f}.'.format(np.mean(test_df['question_text'].apply(lambda x: len(x)))))

Average character length of questions in train is 71.
Average character length of questions in test is 71.


In [5]:
# def clean_text(x):
#     x = str(x)
#     for punct in puncts:
#         x = x.replace(punct, f' {punct} ')
#     return x

# puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
#  '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
#  '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
#  '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
#  '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]



In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_features = 100000

tokenizer = Tokenizer(num_words=max_features, lower=True)
full_text = list(train_df['question_text'].values) + list(test_df['question_text'].values)
tokenizer.fit_on_texts(full_text)

train_tokenized = tokenizer.texts_to_sequences(train_df['question_text'].fillna('missing'))
test_tokenized = tokenizer.texts_to_sequences(test_df['question_text'].fillna('missing'))

Using TensorFlow backend.


In [7]:
print(len(train_tokenized))

print(type(train_tokenized[2]))
print(train_tokenized[2])
print(train_df.question_text[2])
print(train_tokenized[50])
print(train_df.question_text[50])


1306122
<class 'list'>
[16, 26, 1995, 375, 70, 26, 1995, 375, 450, 5341]
Why does velocity affect time? Does velocity affect space geometry?
[2, 12, 38, 222, 5, 13511, 28, 610, 10, 2, 12, 1, 1722, 7, 208, 17]
What are some ways to shorten your period, and what are the risks of doing it?


In [8]:


max_len = 72
train_X = pad_sequences(train_tokenized, maxlen=max_len)
test_X = pad_sequences(test_tokenized, maxlen=max_len)
train_y = train_df.target.values

print(train_X[2])
print(train_X.shape)
# np.random.seed(0)
# train_indices = np.random.permutation(len(train_X))

# train_X = train_X[train_indices]
# train_y = train_y[train_indices]

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0   16   26 1995  375   70   26 1995  375
  450 5341]
(1306122, 72)


In [9]:
from gensim.models import KeyedVectors
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec

glove_path= '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
news_path = '../input/embeddings/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'

embeddings = KeyedVectors.load_word2vec_format(news_path, binary=True)
print(embeddings.vectors.shape)

(3000000, 300)


In [10]:
print((embeddings.get_vector('hello')))

[-0.05419922  0.01708984 -0.00527954  0.33203125 -0.25       -0.01397705
 -0.15039062 -0.265625    0.01647949  0.3828125  -0.03295898 -0.09716797
 -0.16308594 -0.04443359  0.00946045  0.18457031  0.03637695  0.16601562
  0.36328125 -0.25585938  0.375       0.171875    0.21386719 -0.19921875
  0.13085938 -0.07275391 -0.02819824  0.11621094  0.15332031  0.09082031
  0.06787109 -0.0300293  -0.16894531 -0.20800781 -0.03710938 -0.22753906
  0.26367188  0.012146    0.18359375  0.31054688 -0.10791016 -0.19140625
  0.21582031  0.13183594 -0.03515625  0.18554688 -0.30859375  0.04785156
 -0.10986328  0.14355469 -0.43554688 -0.0378418   0.10839844  0.140625
 -0.10595703  0.26171875 -0.17089844  0.39453125  0.12597656 -0.27734375
 -0.28125     0.14746094 -0.20996094  0.02355957  0.18457031  0.00445557
 -0.27929688 -0.03637695 -0.29296875  0.19628906  0.20703125  0.2890625
 -0.20507812  0.06787109 -0.43164062 -0.10986328 -0.2578125  -0.02331543
  0.11328125  0.23144531 -0.04418945  0.10839844 -0.28

In [11]:
print(len(tokenizer.word_index))
mean = np.mean(embeddings.vectors)
std = np.std(embeddings.vectors)
print(mean)
print(std)

256198
-0.003527845
0.13315111


In [12]:
_, size = embeddings.vectors.shape
print(size)
print(len(tokenizer.word_index))
num_words = min(len(tokenizer.word_index), max_features)
embedding_matrix = np.random.normal(mean, std, (num_words, size))
print(embedding_matrix.shape)
# print(embedding_matrix[0])

300
256198
(100000, 300)


In [13]:
print(type(tokenizer.word_index))

<class 'dict'>


In [14]:
for word, index in tokenizer.word_index.items():
    if index >= max_features: continue
    try:
        word_vector = embeddings.get_vector(word)
        embedding_matrix[index] = word_vector
    except:
        pass
print(embedding_matrix.shape)

(100000, 300)


In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from tqdm import  tqdm
tqdm.pandas()

def seed_torch(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed = 0
seed_torch(seed)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(max_features, size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        
        self.conv_0 = nn.Conv2d(in_channels=1, out_channels=100, kernel_size=(3, size))
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=100, kernel_size=(4, size))
        self.conv_2 = nn.Conv2d(in_channels=1, out_channels=100, kernel_size=(5, size))
        self.fc = nn.Linear(3 * 100, 1)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, text):
#         print('text',text.shape)
#         print('after',text.shape)
        embedded = self.embedding(text)
#         print('embedded', embedded.shape)
        embedded = embedded.unsqueeze(1)
        conved_0 = F.relu(self.conv_0(embedded).squeeze(3))
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))
        conved_2 = F.relu(self.conv_2(embedded).squeeze(3))
        
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim=1))
        return self.fc(cat)
        
        
    

In [16]:
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader, TensorDataset
import time
import re
import random
from sklearn.metrics import f1_score

splits = list(StratifiedKFold(n_splits=4, shuffle=True, random_state=0).split(train_X, train_y))
train_epochs = 5
batch_size = 64

print(type(train_X), train_X.shape)
print(type(train_y), train_y.shape)
train_X = torch.Tensor(train_X)
train_y = torch.Tensor(train_y)

train_preds = np.zeros((len(train_X)))
test_preds = np.zeros((len(test_X)))

x_test_cuda = torch.tensor(test_X, dtype=torch.long).cuda()
test = torch.utils.data.TensorDataset(x_test_cuda)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)

for i, (train_idx, valid_idx) in enumerate(splits):
    print('Fold:{0}'.format(i))
    x_train_fold = torch.tensor(train_X[train_idx], dtype=torch.long).cuda()
    y_train_fold = torch.tensor(train_y[train_idx, np.newaxis], dtype=torch.float32).cuda()
    x_val_fold = torch.tensor(train_X[valid_idx], dtype=torch.long).cuda()
    y_val_fold = torch.tensor(train_y[valid_idx, np.newaxis], dtype=torch.float32).cuda()
    
    train_data = torch.utils.data.TensorDataset(x_train_fold, y_train_fold)
    valid_data = torch.utils.data.TensorDataset(x_val_fold, y_val_fold)
    
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, shuffle=False)
    
    model = CNN()
    model.cuda()
    loss_function = nn.BCEWithLogitsLoss()
    optimiser = optim.Adam(model.parameters())
    
    for epoch in range(train_epochs):
        start_time = time.time()
        model.train()
        avg_loss = 0.0
        # train  model
        for x_batch, y_batch in tqdm(train_loader, disable=True):
#             print(x_batch.shape)
#             print(y_batch.shape)
            y_predicted = model(x_batch)
#             print(y_predicted.shape)
#             print(y_predicted[0])
            loss = loss_function(y_predicted, y_batch)
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
            avg_loss += loss.item()/len(train_loader)

    # evaluate model
        model.eval()
        val_predicted_fold = np.zeros(x_val_fold.size(0))
        test_predicted_fold = np.zeros(len(test_X))
        avg_val_loss =0.0

        for i, (x_batch, y_batch) in enumerate(valid_loader):
            y_predicted = model(x_batch).detach()
            avg_val_loss += loss_function(y_predicted, y_batch).item()/len(valid_loader)
            val_predicted_fold[i*batch_size:(i+1)*batch_size] = sigmoid(y_predicted.cpu().numpy())[:,0]   

        elapsed_time = time.time() - start_time
        print('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f} \t time={:.2f}s'.format(epoch + 1,
                                                        train_epochs, avg_loss, avg_val_loss, elapsed_time))
    # test model on testing dataset
    test_predicted_fold = np.zeros(len(test_X))
    for i, (x_batch,) in enumerate(test_loader):
        y_pred = model(x_batch).detach()
        test_predicted_fold[i * batch_size:(i+1) * batch_size] = sigmoid(y_pred.cpu().numpy())[:, 0]
    
    train_preds[valid_idx] = val_predicted_fold
    test_preds += test_predicted_fold / len(splits) 

<class 'numpy.ndarray'> (1306122, 72)
<class 'numpy.ndarray'> (1306122,)
Fold:0


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(Tru

Epoch 1/5 	 loss=0.1206 	 val_loss=0.1146 	 time=84.12s
Epoch 2/5 	 loss=0.1086 	 val_loss=0.1110 	 time=83.78s
Epoch 3/5 	 loss=0.1003 	 val_loss=0.1113 	 time=83.97s
Epoch 4/5 	 loss=0.0911 	 val_loss=0.1157 	 time=83.65s
Epoch 5/5 	 loss=0.0824 	 val_loss=0.1192 	 time=83.68s
Fold:1
Epoch 1/5 	 loss=0.1207 	 val_loss=0.1202 	 time=83.79s
Epoch 2/5 	 loss=0.1087 	 val_loss=0.1105 	 time=84.05s
Epoch 3/5 	 loss=0.1003 	 val_loss=0.1116 	 time=83.74s
Epoch 4/5 	 loss=0.0917 	 val_loss=0.1149 	 time=83.72s
Epoch 5/5 	 loss=0.0835 	 val_loss=0.1215 	 time=84.22s
Fold:2
Epoch 1/5 	 loss=0.1208 	 val_loss=0.1120 	 time=83.78s
Epoch 2/5 	 loss=0.1091 	 val_loss=0.1106 	 time=83.88s
Epoch 3/5 	 loss=0.1008 	 val_loss=0.1102 	 time=83.97s
Epoch 4/5 	 loss=0.0925 	 val_loss=0.1147 	 time=84.43s
Epoch 5/5 	 loss=0.0837 	 val_loss=0.1196 	 time=83.92s
Fold:3
Epoch 1/5 	 loss=0.1208 	 val_loss=0.1119 	 time=83.84s
Epoch 2/5 	 loss=0.1093 	 val_loss=0.1097 	 time=84.40s
Epoch 3/5 	 loss=0.1012 	 v

In [17]:
def threshold_search(y_true, y_proba):
    best_threshold = 0
    best_score = 0
    for threshold in tqdm([i * 0.01 for i in range(100)]):
        score = f1_score(y_true=y_true, y_pred=y_proba > threshold)
        if score > best_score:
            best_threshold = threshold
            best_score = score
    search_result = {'threshold': best_threshold, 'f1': best_score}
    return search_result

In [18]:
search_result = threshold_search(train_y, train_preds)
search_result

100%|██████████| 100/100 [00:29<00:00,  3.26it/s]


{'threshold': 0.26, 'f1': 0.6311257860553635}

In [19]:
sub = pd.read_csv('../input/sample_submission.csv')
sub.prediction = test_preds > search_result['threshold']
sub.to_csv("submission.csv", index=False)

In [20]:
# import operator 
# def check_coverage(vocab,embeddings):
#     a = {}
#     oov = {}
#     k = 0
#     i = 0
#     for word in tqdm(vocab):
#         try:
#             a[word] = embeddings[word]
#             k += vocab[word]
#         except:

#             oov[word] = vocab[word]
#             i += vocab[word]
#             pass

#     print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
#     print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
#     sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]
#     return sorted_x